Descarga, extracción y análisis del cuestionario ampliado Viviendas del Censo 2020 de la Ciudad de México utilizando Winrar para obtener datos del agua en las viviendas

In [3]:
#Instalar paquetería

if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse,srvyr, gt, showtext,janitor,sf,httr,jsonlite)
font_add_google("Poppins", "pop")
showtext_auto()

Cargando paquete requerido: pacman



In [4]:
##Borrar datos del entorno
rm(list=ls())

In [5]:
#Opción para que la descarga no se corte
options(timeout=1600)

In [6]:
#URL del Censo 2020
url<-"https://www.inegi.org.mx/contenidos/programas/ccpv/2020/microdatos/Censo2020_CA_cdmx_csv.zip"

In [7]:
# Establecer directorio de trabajo y crear carpeta de extracción en "Mis Documentos"
raiz <- file.path(Sys.getenv("USERPROFILE"), "Documents")
dir.create(file.path(raiz, "datoscenso"), showWarnings = F)
directorio <- file.path(raiz, "datoscenso")
setwd(directorio)

In [8]:
#Se establece que, si no existe el archivo viviendas, se descargue el zip de la URL de INEGI
if (!file.exists("Viviendas09.CSV")){
  download.file(url,destfile = "censo.zip",quiet = FALSE, mode="wb")
  #Usaremos Winrar como opción para extracción
  winrar=shQuote("C:/Program Files/WinRAR/WinRAR")
  file="censo.zip"
  dest=directorio
  #Ejecución de la extracción
  cmd=paste(winrar, ' e ',file, ' -ir!*.*','"',dest,'"',sep='')
  system(cmd)
  unlink("datoscenso/censo.zip")
}

In [9]:
# Listar todos los archivos y carpetas dentro del directorio de "datoscenso"
contenido <- list.files(path = directorio, recursive = TRUE, full.names = TRUE)

# Mostrar el contenido
print(contenido)


[1] "C:\\Users\\quetz/Documents/datoscenso/censo.zip"      
[2] "C:\\Users\\quetz/Documents/datoscenso/Migrantes09.CSV"
[3] "C:\\Users\\quetz/Documents/datoscenso/Personas09.CSV" 
[4] "C:\\Users\\quetz/Documents/datoscenso/Viviendas09.CSV"


In [10]:
# Abrir el archivo "Viviendas09.CSV"
archivo_viviendas <- file.path(directorio, "Viviendas09.CSV")

In [11]:
# Verificar si el archivo existe antes de intentar abrirlo
if (file.exists(archivo_viviendas)) {
  # Leer el archivo CSV
  viviendas <- read.csv(archivo_viviendas, header = TRUE, sep = ",", stringsAsFactors = FALSE)
  
  # Mostrar las primeras filas del archivo para verificar la carga
  print(head(viviendas))
} else {
  cat("El archivo Viviendas09.CSV no se encuentra en el directorio.")
  
}


  ENT MUN LOC50K    ID_VIV COBERTURA        ESTRATO UPM FACTOR CLAVIVP PAREDES
1   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
2   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
3   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
4   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
5   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
6   9   2      1 9.002e+10         2 09-002-0001-00   1     27       1       8
  TECHOS PISOS COCINA CUADORM TOTCUART LUG_COC COMBUSTIBLE ESTUFA ELECTRICIDAD
1     10     3      1       3        4       1           2     NA            1
2     10     3      1       5       10       1           2     NA            1
3     10     3      1       4        7       1           2     NA            1
4     10     2      1       3        4       1           2     NA            1
5     10     2      1       4        5       1      

In [12]:
# Definir diseño muestral con nest=TRUE
dm <- viviendas %>%
  as_survey(weights = FACTOR, strata = ESTRATO, ids = UPM, nest = TRUE)

In [13]:
# Evitar notación científica y mostrar números con dos decimales
options(scipen = 999, digits = 2)

In [15]:
###--- Calcular el total de Viviendas en la Ciudad de México
print("Total de viviendas particulares habitadas en la Ciudad de México")
dm%>%
  summarise(viviendas=survey_total())

[1] "Total de viviendas particulares habitadas en la Ciudad de México"


viviendas,viviendas_se
<dbl>,<dbl>
2720459,76433


In [23]:
# Pregunta del censo: AGUA_ENTUBADA


#¿El agua la obtienen de llaves o mangueras que están: dentro de la vivienda?
#sólo en el patio o terreno? ¿No tienen agua entubada?

dm %>%
  mutate(AGUA_ENTUBADA = case_when(
    AGUA_ENTUBADA == 1 ~ "Dentro de la Vivienda",
    AGUA_ENTUBADA == 2 ~ "Sólo en el patio o terreno",
    AGUA_ENTUBADA == 3 ~ "No tienen agua entubada",
    is.na(AGUA_ENTUBADA) ~ "No contestó",  # Corregido
    AGUA_ENTUBADA == 9 ~ "No especificó"
  )) %>%
  group_by(AGUA_ENTUBADA) %>%
  summarise(viviendas = survey_total() / 1000000,
            pct = survey_prop()) %>%
  janitor::adorn_totals()

print("El 89.73% de las viviendas de la Ciudad de México tienen agua entubada por dentro de ellas")



AGUA_ENTUBADA,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Dentro de la Vivienda,2.44129,0.07211,0.89738,0.00524
No contestó,0.01008,0.00194,0.00371,0.00071
No especificó,0.00051,0.00029,0.00019,0.00011
No tienen agua entubada,0.02406,0.00329,0.00884,0.00121
Sólo en el patio o terreno,0.24452,0.01452,0.08988,0.00500
Total,2.72046,0.09214,1.00000,0.01227


[1] "El 89.73% de las viviendas de la Ciudad de México tienen agua entubada por dentro de ellas"


In [31]:
# Pregunta del censo: ABA_AGUA_ENTU

#¿El agua que usan en su vivienda proviene: del servicio público de agua? 
#de un pozo comunitario? de un pozo particular? de una pipa? de otra vivienda? de la lluvia? de otro lugar?

dm%>%
  mutate(ABA_AGUA_ENTU=case_when(
    ABA_AGUA_ENTU==1 ~ "Del servicio público de agua",
    ABA_AGUA_ENTU==2 ~ "De un pozo comunitario",
    ABA_AGUA_ENTU==3 ~ "De un pozo particular",
    ABA_AGUA_ENTU==4 ~ "De una pipa",
    ABA_AGUA_ENTU==5 ~ "De otra vivienda",
    ABA_AGUA_ENTU==6 ~ "De la lluvia",
    ABA_AGUA_ENTU==7 ~ "De otra vivienda",
    ABA_AGUA_ENTU==9 ~ "No especificó",))%>%
  group_by(ABA_AGUA_ENTU)%>%
  summarise(viviendas=survey_total()/1000000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 96.07% de las viviendas de la Ciudad de México tienen agua entubada del servicio público del agua")

ABA_AGUA_ENTU,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
De la lluvia,0.00075,0.00044,0.00027,0.00016
De otra vivienda,0.00395,0.00064,0.00145,0.00023
De un pozo comunitario,0.01626,0.00345,0.00598,0.00127
De un pozo particular,0.00132,0.00037,0.00048,0.00014
De una pipa,0.04812,0.00824,0.01769,0.00297
Del servicio público de agua,2.61373,0.07481,0.96077,0.00385
No especificó,0.00168,0.00036,0.00062,0.00013
NA,0.03465,0.00383,0.01274,0.00142
Total,2.72046,0.09215,1.00000,0.01017


[1] "El 96.07% de las viviendas de la Ciudad de México tienen agua entubada del servicio público del agua"


In [32]:
# Pregunta del censo: ABA_AGUA_NO_ENTU

#Entonces, ¿acarrean el agua de: un pozo? una llave comunitaria? otra vivienda? 
#un río, arroyo o lago? ¿La trae una pipa? ¿La captan de la lluvia?

dm%>%
  mutate(ABA_AGUA_NO_ENTU=case_when(
    ABA_AGUA_NO_ENTU==1 ~ "Un pozo",
    ABA_AGUA_NO_ENTU==2 ~ "Una llave comunitariao",
    ABA_AGUA_NO_ENTU==3 ~ "Otra vivienda",
    ABA_AGUA_NO_ENTU==4 ~ "Un río, arroyo o lago",
    ABA_AGUA_NO_ENTU==5 ~ "La trae una pipa",
    ABA_AGUA_NO_ENTU==6 ~ "La captan de la lluvia",
    ABA_AGUA_NO_ENTU==9 ~ "No especificó",))%>%
  group_by(ABA_AGUA_NO_ENTU)%>%
  summarise(viviendas=survey_total()/1000000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()


ABA_AGUA_NO_ENTU,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
La captan de la lluvia,0.00022,0.000080,0.000081,0.000029
La trae una pipa,0.01605,0.002820,0.005900,0.001042
Otra vivienda,0.00222,0.000327,0.000816,0.000118
Un pozo,0.00206,0.000701,0.000757,0.000258
"Un río, arroyo o lago",0.00015,0.000098,0.000057,0.000036
Una llave comunitariao,0.00335,0.000697,0.001231,0.000257
NA,2.69640,0.076240,0.991157,0.001213
Total,2.72046,0.080964,1.000000,0.002953


In [33]:
library(dplyr)
library(janitor)
library(glue)

# Calcular el porcentaje y total de viviendas para cada categoría
resultados <- dm %>%
  mutate(ABA_AGUA_NO_ENTU = case_when(
    ABA_AGUA_NO_ENTU == 1 ~ "Un pozo",
    ABA_AGUA_NO_ENTU == 2 ~ "Una llave comunitaria",
    ABA_AGUA_NO_ENTU == 3 ~ "Otra vivienda",
    ABA_AGUA_NO_ENTU == 4 ~ "Un río, arroyo o lago",
    ABA_AGUA_NO_ENTU == 5 ~ "La trae una pipa",
    ABA_AGUA_NO_ENTU == 6 ~ "La captan de la lluvia",
    ABA_AGUA_NO_ENTU == 9 ~ "No especificó"
  )) %>%
  group_by(ABA_AGUA_NO_ENTU) %>%
  summarise(
    viviendas = survey_total() / 1000000,  # Total de viviendas
    pct = survey_prop()                   # Porcentaje
  ) %>%
  janitor::adorn_totals()

# Verificar si la creación de resultados fue exitosa
if (exists("resultados")) {
  # Sumar los porcentajes para las fuentes alternativas excluyendo NA
  suma_sin_na <- resultados %>%
    filter(ABA_AGUA_NO_ENTU != "No especificó") %>%
    summarise(suma_pct_sin_na = sum(pct, na.rm = TRUE))

  # Mostrar los porcentajes individuales de cada categoría con total de viviendas
  resultados %>%
    mutate(
      pct = round(pct * 100, 4),  # Redondear porcentaje
      viviendas = round(viviendas, 4),  # Redondear total de viviendas
      print_msg = glue("{ABA_AGUA_NO_ENTU}: {pct}% ({viviendas} viviendas)")
    ) %>%
    arrange(desc(pct)) %>%
    pull(print_msg) %>%
    walk(print)

  # Mostrar el porcentaje acumulado excluyendo "No especificó"
  print(glue("\nSuma de los porcentajes (sin 'No especificó'): {round(suma_sin_na$suma_pct_sin_na * 100, 4)}%"))
} else {
  print("No se pudo crear el objeto 'resultados'.")
}


Total: 100% (2.7205 viviendas)
NA: 99.1157% (2.6964 viviendas)
La trae una pipa: 0.59% (0.0161 viviendas)
Una llave comunitaria: 0.1231% (0.0034 viviendas)
Otra vivienda: 0.0816% (0.0022 viviendas)
Un pozo: 0.0757% (0.0021 viviendas)
La captan de la lluvia: 0.0081% (0.0002 viviendas)
Un río, arroyo o lago: 0.0057% (0.0002 viviendas)
Suma de los porcentajes (sin 'No especificó'): 100.8843%


In [34]:
# Pregunta del censo: TINACO

#¿En esta vivienda tienen: tinaco?

dm%>%
  mutate(TINACO=case_when(
    TINACO==1 ~ "Sí",
    TINACO==2 ~ "No",
    TINACO==9 ~ "No especificó",))%>%
  group_by(TINACO)%>%
  summarise(viviendas=survey_total()/1000000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

  print("El 82.84% de las viviendas de la Ciudad de México tienen tinaco")

TINACO,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
No,0.4550,0.0281,0.16727,0.00866
No especificó,0.0016,0.0006,0.00059,0.00022
Sí,2.2537,0.0649,0.82844,0.00864
NA,0.0101,0.0019,0.00371,0.00071
Total,2.7205,0.0955,1.00000,0.01823


[1] "El 82.84% de las viviendas de la Ciudad de México tienen tinaco"


In [35]:
# Pregunta del censo: CONAGUA

#¿La taza de baño (letrina): tiene descarga directa de agua? le echan agua con cubeta? ¿No se le puede echar agua?

dm%>%
  mutate(CONAGUA=case_when(
    CONAGUA==1 ~ "Tiene descarga directa de agua",
    CONAGUA==2 ~ "Le echan agua con cubeta",
    CONAGUA==3 ~ "No se le puede echar agua",
    CONAGUA==9 ~ "No especificó",))%>%
  group_by(CONAGUA)%>%
  summarise(viviendas=survey_total()/1000000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 86.25% de las viviendas de la Ciudad de México tienen descarga directa de agua en la taza del baño")

CONAGUA,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
Le echan agua con cubeta,0.35532,0.01756,0.1306,0.005724
No especificó,0.00028,0.00013,0.0001,0.000048
No se le puede echar agua,0.00274,0.00047,0.0010,0.000173
Tiene descarga directa de agua,2.34629,0.06962,0.8625,0.005836
NA,0.01583,0.00204,0.0058,0.000747
Total,2.72046,0.08983,1.0000,0.012529


[1] "El 86.25% de las viviendas de la Ciudad de México tienen descarga directa de agua en la taza del baño"


In [36]:
# Pregunta del censo: DRENAJE

#¿Esta vivienda tiene drenaje o desagüe conectado a: la red pública? una fosa séptica o 
#tanque séptico (biodigestor)? una tubería que va a dar a una barranca o grieta? 
#una tubería que va a dar a un río, lago o mar? ¿No tiene drenaje?

dm%>%
  mutate(DRENAJE=case_when(
    DRENAJE==1 ~ "La red pública",
    DRENAJE==2 ~ "Una fosa séptica o tanque séptico (biodigestor)",
    DRENAJE==3 ~ "Una tubería que va a dar a una barranca o grieta",
    DRENAJE==4 ~ "Una tubería que va a dar a un río, lago o mar",
    DRENAJE==5 ~ "No tiene drenaje",
    DRENAJE==9 ~ "No especificado",))%>%
  group_by(DRENAJE)%>%
  summarise(viviendas=survey_total()/1000000,
            pct=survey_prop()) %>%
  janitor::adorn_totals()

print("El 94.12% de las viviendas de la Ciudad de México tienen drenaje a la red pública")

DRENAJE,viviendas,viviendas_se,pct,pct_se
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
La red pública,2.56051,0.07476,0.94121,0.004987
No especificado,0.00065,0.00036,0.00024,0.000131
No tiene drenaje,0.00430,0.00062,0.00158,0.000227
Una fosa séptica o tanque séptico (biodigestor),0.13628,0.01325,0.05009,0.004802
"Una tubería que va a dar a un río, lago o mar",0.00038,0.00014,0.00014,0.000051
Una tubería que va a dar a una barranca o grieta,0.00826,0.00163,0.00304,0.000600
NA,0.01008,0.00194,0.00371,0.000712
Total,2.72046,0.09269,1.00000,0.011510


[1] "El 94.12% de las viviendas de la Ciudad de México tienen drenaje a la red pública"
